In [1]:
import numpy as np
import pandas as pd
import os
os.chdir('C:\\Users\\Ben\\OneDrive - Duke University\\bilbo_lab\\il34_project\\il34_dreadds_rnascope\\RNAScope\\il34_dreadds_rnascope_gad2_fos\\imaris\\amelie_RNAFISH_rd2')

In [2]:
data_files = [i for i in os.listdir() if 'G00' in i]
final_df = pd.DataFrame(columns = ['index', 'Fos_puncta_adj', 'Gad2_puncta_adj', 'IL34_puncta_adj', 'fos_status', 'cell_type'])
total_il34 = {}

for file in data_files:
    fos_df = pd.read_excel(file, sheet_name = 1, header = 1)[['Cell Number Of Vesicles', 'ID']].set_index('ID').rename(columns = {'Cell Number Of Vesicles' : 'Fos_puncta'})
    gad2_df = pd.read_excel(file, sheet_name = 2, header = 1)[['Cell Number Of Vesicles', 'ID']].set_index('ID').rename(columns = {'Cell Number Of Vesicles' : 'Gad2_puncta'})
    il34_df = pd.read_excel(file, sheet_name = 3, header = 1)[['Cell Number Of Vesicles', 'ID']].set_index('ID').rename(columns = {'Cell Number Of Vesicles' : 'IL34_puncta'})
    cell_vol_df = pd.read_excel(file, sheet_name = 4, header = 1).set_index('ID')['Cell Volume']
    summary_df = pd.read_excel(file, sheet_name = 0, header = 1).set_index('Variable')
    
    
    combo_df = fos_df.join(gad2_df).join(il34_df).join(cell_vol_df)
    
    combo_df = combo_df[combo_df['Cell Volume'] > 200]
    combo_df.reset_index(inplace = True)
    
    combo_df['Fos_puncta_adj'] = 500*(combo_df['Fos_puncta'] / combo_df['Cell Volume'])
    combo_df['Gad2_puncta_adj'] = 500*(combo_df['Gad2_puncta'] / combo_df['Cell Volume'])
    combo_df['IL34_puncta_adj'] = 500*(combo_df['IL34_puncta'] / combo_df['Cell Volume'])
#     fig1 = combo_df[['Gad2_puncta_adj', 'Fos_puncta_adj', 'IL34_puncta_adj']].plot(figsize=(25,8), 
#                                                                    kind = 'bar', 
#                                                                    title = file.split('.')[0], 
#                                                                    ylabel = '# of Puncta / 500um3').get_figure();
    
    
    neuron_df = combo_df
    
    fos_status = ['fos-']*len(neuron_df)
    cell_type = ['VGlut1']*len(neuron_df)

    for row in range(len(neuron_df)):

        if neuron_df['Fos_puncta_adj'][row] > 6:
            fos_status[row] = 'fos+'
        elif neuron_df['Fos_puncta_adj'][row] < 6:
            fos_status[row] = 'fos-'

        neuron_df['fos_status'] = fos_status


        if neuron_df['Gad2_puncta_adj'][row] > 7:
            cell_type[row] = 'Gad2'
        elif neuron_df['Gad2_puncta_adj'][row] < 7:
            if neuron_df['IL34_puncta_adj'][row] > 5:
                cell_type[row] = 'VGlut1'
            elif neuron_df['IL34_puncta_adj'][row] < 6:
                cell_type[row] = 'nan'
                
        neuron_df['cell_type'] = cell_type
        

    neuron_df['index'] = file
    neuron_df = neuron_df[['index', 'Fos_puncta_adj', 'Gad2_puncta_adj', 'IL34_puncta_adj', 'fos_status', 'cell_type']]
    neuron_df = neuron_df[neuron_df['cell_type'] != 'nan']
    final_df = final_df.append(neuron_df)
    final_df['IL34_puncta_adj'] = final_df['IL34_puncta_adj'].astype(float)

In [3]:
final_df

,index,Fos_puncta_adj,Gad2_puncta_adj,IL34_puncta_adj,fos_status,cell_type
1,D05_A01_G001_0001.xls,9.961177,51.051030,6.225735,fos+,Gad2
2,D05_A01_G001_0001.xls,1.882204,5.646612,11.293225,fos-,VGlut1
3,D05_A01_G001_0001.xls,6.639031,2.655612,18.589285,fos+,VGlut1
4,D05_A01_G001_0001.xls,1.417033,38.259883,14.170327,fos-,Gad2
5,D05_A01_G001_0001.xls,2.554448,0.000000,16.603912,fos-,VGlut1
...,...,...,...,...,...,...
78,D22_A01_G004_0001.xls,0.000000,0.000000,6.003266,fos-,VGlut1
79,D22_A01_G004_0001.xls,0.000000,0.000000,20.764058,fos-,VGlut1
81,D22_A01_G004_0001.xls,0.000000,0.000000,16.908572,fos-,VGlut1
83,D22_A01_G004_0001.xls,0.000000,0.000000,13.775285,fos-,VGlut1


In [4]:
image_number = []
animal_id = []

for i in final_df['index']:
    image_number = np.append(image_number, i.split('_')[2].split('G00')[1])
    animal_id = np.append(animal_id, i.split('_')[0])
    
final_df['animal_id'] = animal_id
final_df['image_number'] = image_number

In [6]:
final_df.to_csv('final_df_raw.csv')

In [94]:
#final_df.groupby(['animal_id', 'fos_status', 'cell_type'])['IL34_puncta_adj'].mean().to_csv('grouped_animal_output.csv')

In [103]:
#final_df.groupby(['index', 'fos_status', 'cell_type', 'animal_id']).count().groupby(['animal_id', 'fos_status', 'cell_type'])['image_number'].mean().to_csv('cell_counts.csv')

# Now I need to make sure I analyze the original 4 in the exact same way...

In [6]:
original_files = [i for i in os.listdir() if 'image' in i]

og_final_df = pd.DataFrame(columns = ['index', 'Fos_puncta_adj', 'Gad2_puncta_adj', 'IL34_puncta_adj', 'fos_status', 'cell_type'])
total_il34 = {}

for file in original_files:
    gad2_df = pd.read_excel(file, sheet_name = 1, header = 1)[['Cell Number Of Vesicles', 'ID']].set_index('ID').rename(columns = {'Cell Number Of Vesicles' : 'Gad2_puncta'})
    fos_df = pd.read_excel(file, sheet_name = 2, header = 1)[['Cell Number Of Vesicles', 'ID']].set_index('ID').rename(columns = {'Cell Number Of Vesicles' : 'Fos_puncta'})
    il34_df = pd.read_excel(file, sheet_name = 3, header = 1)[['Cell Number Of Vesicles', 'ID']].set_index('ID').rename(columns = {'Cell Number Of Vesicles' : 'IL34_puncta'})
    cell_vol_df = pd.read_excel(file, sheet_name = 4, header = 1).set_index('ID')['Cell Volume']
    summary_df = pd.read_excel(file, sheet_name = 0, header = 1).set_index('Variable')
    
    
    combo_df = fos_df.join(gad2_df).join(il34_df).join(cell_vol_df)
    
    combo_df = combo_df[combo_df['Cell Volume'] > 200]
    combo_df.reset_index(inplace = True)
    
    combo_df['Fos_puncta_adj'] = 500*(combo_df['Fos_puncta'] / combo_df['Cell Volume'])
    combo_df['Gad2_puncta_adj'] = 500*(combo_df['Gad2_puncta'] / combo_df['Cell Volume'])
    combo_df['IL34_puncta_adj'] = 500*(combo_df['IL34_puncta'] / combo_df['Cell Volume'])
#     fig1 = combo_df[['Gad2_puncta_adj', 'Fos_puncta_adj', 'IL34_puncta_adj']].plot(figsize=(25,8), 
#                                                                    kind = 'bar', 
#                                                                    title = file.split('.')[0], 
#                                                                    ylabel = '# of Puncta / 500um3').get_figure();
    
    
    neuron_df = combo_df
    
    fos_status = ['fos-']*len(neuron_df)
    cell_type = ['VGlut1']*len(neuron_df)

    for row in range(len(neuron_df)):

        if neuron_df['Fos_puncta_adj'][row] > 6:
            fos_status[row] = 'fos+'
        elif neuron_df['Fos_puncta_adj'][row] < 6:
            fos_status[row] = 'fos-'

        neuron_df['fos_status'] = fos_status


        if neuron_df['Gad2_puncta_adj'][row] > 7:
            cell_type[row] = 'Gad2'
        elif neuron_df['Gad2_puncta_adj'][row] < 7:
            if neuron_df['IL34_puncta_adj'][row] > 5:
                cell_type[row] = 'VGlut1'
            elif neuron_df['IL34_puncta_adj'][row] < 6:
                cell_type[row] = 'nan'
                
        neuron_df['cell_type'] = cell_type
        

    neuron_df['index'] = file
    neuron_df = neuron_df[['index', 'Fos_puncta_adj', 'Gad2_puncta_adj', 'IL34_puncta_adj', 'fos_status', 'cell_type']]
    neuron_df = neuron_df[neuron_df['cell_type'] != 'nan']
    og_final_df = og_final_df.append(neuron_df)
    og_final_df['IL34_puncta_adj'] = og_final_df['IL34_puncta_adj'].astype(float)

In [7]:
image_number = []
animal_id = []

for i in og_final_df['index']:
    image_number = np.append(image_number, i.split(' ')[4].split('.')[0])
    animal_id = np.append(animal_id, i.split('_')[0])
    
og_final_df['animal_id'] = animal_id
og_final_df['image_number'] = image_number

In [8]:
combined_final_df = pd.concat([final_df, og_final_df])

In [10]:
combined_final_df

,index,Fos_puncta_adj,Gad2_puncta_adj,IL34_puncta_adj,fos_status,cell_type,animal_id,image_number
1,D05_A01_G001_0001.xls,9.961177,51.051030,6.225735,fos+,Gad2,D05,1
2,D05_A01_G001_0001.xls,1.882204,5.646612,11.293225,fos-,VGlut1,D05,1
3,D05_A01_G001_0001.xls,6.639031,2.655612,18.589285,fos+,VGlut1,D05,1
4,D05_A01_G001_0001.xls,1.417033,38.259883,14.170327,fos-,Gad2,D05,1
5,D05_A01_G001_0001.xls,2.554448,0.000000,16.603912,fos-,VGlut1,D05,1
...,...,...,...,...,...,...,...,...
227,d04_f_a_d04_f_1 image 1_e_d04_f_5 image 5.xls,2.317025,4.634049,6.951074,fos-,VGlut1,d04,5
228,d04_f_a_d04_f_1 image 1_e_d04_f_5 image 5.xls,4.631742,19.684902,3.473806,fos-,Gad2,d04,5
231,d04_f_a_d04_f_1 image 1_e_d04_f_5 image 5.xls,1.839757,0.919879,5.519272,fos-,VGlut1,d04,5
232,d04_f_a_d04_f_1 image 1_e_d04_f_5 image 5.xls,2.204498,11.022490,2.204498,fos-,Gad2,d04,5


In [11]:
combined_final_df.groupby(['animal_id', 'fos_status', 'cell_type'])['IL34_puncta_adj'].mean()

animal_id  fos_status  cell_type
D05        fos+        Gad2          8.921079
                       VGlut1       14.688447
           fos-        Gad2          7.677604
                       VGlut1       13.782197
D06        fos+        Gad2          6.087531
                                      ...    
d03        fos-        VGlut1        9.235495
d04        fos+        Gad2          6.107105
                       VGlut1        8.420841
           fos-        Gad2          4.665273
                       VGlut1        7.909483
Name: IL34_puncta_adj, Length: 88, dtype: float64

In [135]:
combined_final_df.groupby(['animal_id', 'fos_status', 'cell_type'])['IL34_puncta_adj'].mean().to_csv('grouped_animal_output.csv')
combined_final_df.groupby(['index', 'fos_status', 'cell_type', 'animal_id']).count().groupby(['animal_id', 'fos_status', 'cell_type'])['image_number'].mean().to_csv('cell_counts.csv')